In [ ]:
!pip install -q haystack-ai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
from haystack.components.writers import DocumentWriter
from haystack.components.converters import MarkdownToDocument, PyPDFToDocument, TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore

In [ ]:
import getpass, os


os.environ["HF_API_TOKEN"] = getpass.getpass("Your Hugging Face token")

Your Hugging Face token··········


##Indexing Pipeline

In [ ]:
document_store=InMemoryDocumentStore()
pdf_converter = PyPDFToDocument()
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_by="word", split_length=150, split_overlap=50)
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)


In [ ]:

document_processing_pipeline = Pipeline()
document_processing_pipeline.add_component("converter", pdf_converter)
document_processing_pipeline.add_component("cleaner", document_cleaner)
document_processing_pipeline.add_component("splitter", document_splitter)
document_processing_pipeline.add_component("embedder", document_embedder)
document_processing_pipeline.add_component("writer", document_writer)
document_processing_pipeline.connect("converter", "cleaner")
document_processing_pipeline.connect("cleaner", "splitter")
document_processing_pipeline.connect("splitter", "embedder")
document_processing_pipeline.connect("embedder", "writer")


🚅 Components
  - converter: PyPDFToDocument
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - converter.documents -> cleaner.documents (List[Document])
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> embedder.documents (List[Document])
  - embedder.documents -> writer.documents (List[Document])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:

import gdown

url = "/content/drive/MyDrive/Colab Notebooks/MyDarlingVegan_Clean_Eating_Vegan.pdf"
document_processing_pipeline.run({"converter": {"sources":[url]}})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

{'writer': {'documents_written': 54}}

## RAG Pipeline

In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator

template = [
    ChatMessage.from_user(
        """
Answer the questions based on the given context.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ question }}
Answer:
"""
    )
]

query_pipeline = Pipeline()


query_pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
query_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
query_pipeline.add_component("chat_prompt_builder", ChatPromptBuilder(template=template))
query_pipeline.add_component(
    "llm",
    HuggingFaceAPIChatGenerator(
        api_type="serverless_inference_api", api_params={"model": "HuggingFaceH4/zephyr-7b-beta"}
    ),
)

query_pipeline.connect("embedder.embedding", "retriever.query_embedding")
query_pipeline.connect("retriever", "chat_prompt_builder.documents")
query_pipeline.connect("chat_prompt_builder.prompt", "llm.messages")


🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - chat_prompt_builder: ChatPromptBuilder
  - llm: HuggingFaceAPIChatGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> chat_prompt_builder.documents (List[Document])
  - chat_prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [ ]:
def get_generative_answer(query):

  results =query_pipeline.run({
      "embedder": {"text": query},
      "chat_prompt_builder": {"question": query}
    }
  )
  answer = results["llm"]["replies"][0]
  print(answer)
get_generative_answer("what are TOP 5 CLEAN EATING TIPS")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text="1. Stock your kitchen: Make sure your fridge is full of fruits and vegetables and your pantry is stocked with nuts, seeds, grains, and legumes. This will make clean eating more convenient and less restrictive.\n2. Make a meal plan: Prepare a meal plan for the week and pack snacks for on-the-go. This will help you stay organized and avoid unhealthy choices.\n3. Find healthy comfort foods: Discover a healthy comfort food that you love and make sure to always have it on hand. This will make clean eating more enjoyable and help you stick to it.\n4. Drink lots of water: Always drink plenty of water to help cleanse your system of toxins and maintain a healthy glow.\n5. Carry snacks with you: Whether you're at work, the gym, or running errands, make sure you have clean snacks with you. This will help you avoid unhealthy choices and curb cravings.")], _name=None, _meta={'model': 'HuggingFaceH4/zephyr-7b-beta', 'fi

In [ ]:
question="Can you give me the recipe for how to make DECONSTRUCTED BURRITO BOWLS?"
get_generative_answer(question)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='Ingredients:\n\n- 1 cup uncooked brown rice or quinoa\n- Black Beans (1 cup cooked or 1 ½ cup canned)\n- Avocado Corn Salsa (See recipe below)\n- Mexican Coleslaw (See recipe below)\n- Cilantro (to taste)\n- Hot Sauce (to taste)\n- Cashew Sour Cream (See recipe below)\n\nAvocado Corn Salsa:\n\n- 1 medium-sized avocado, diced\n- 1 cup corn (fresh or canned)\n- 1 small red onion, finely chopped\n- 1 jalapeño pepper, seeded and finely chopped\n- 2 garlic cloves, minced\n- 2 tbsp lime juice\n- Salt (to taste)\n\nMix all ingredients in a medium-sized bowl, adjust seasoning to taste, and refrigerate until ready to use.\n\nMexican Coleslaw:\n\n- 1 medium-sized head of cabbage, thinly sliced\n- 1 small red bell pepper, thinly sliced\n- 1 small red onion, thinly sliced\n- 1 small jalapeño pepper, seeded and finely chopped\n- 3 garlic cloves, minced\n- ¼ cup fresh lime juice\n- 2 tbsp olive oil\n- Salt and pepper (t

In [ ]:
question="what are TOP 5 CLEAN EATING TIPS"
get_generative_answer(question)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text="1. Stock your kitchen: Make sure your fridge is full of fruits and vegetables and your pantry is stocked with nuts, seeds, grains, and legumes. This will make clean eating more convenient and less restrictive.\n2. Make a meal plan: Prepare a meal plan for the week and pack snacks for on-the-go. This will help you stay organized and avoid unhealthy choices.\n3. Find healthy comfort foods: Discover a healthy comfort food that you love and make sure to always have it on hand. This will make clean eating more enjoyable and help you stick to it.\n4. Drink lots of water: Always drink plenty of water to help cleanse your system of toxins and maintain a healthy glow.\n5. Carry snacks with you: Whether you're at work, the gym, or running errands, make sure you have clean snacks with you. This will help you avoid unhealthy choices and curb cravings.")], _name=None, _meta={'model': 'HuggingFaceH4/zephyr-7b-beta', 'fi

## Trying another llm


In [ ]:
query_pipeline.remove_component("llm")
query_pipeline.remove_component("chat_prompt_builder")


Inputs:
  - documents: Any
  - question: Any
  - template: Optional[List[ChatMessage]]
  - template_variables: Optional[Dict[str, Any]]
Outputs:
  - prompt: List[ChatMessage]

In [ ]:
from haystack.components.generators import HuggingFaceAPIGenerator
from haystack.components.builders import PromptBuilder
generator = HuggingFaceAPIGenerator(
    api_type="serverless_inference_api",
    api_params={"model": "mistralai/Mixtral-8x7B-Instruct-v0.1"})
prompt_template = """
According to these documents:

{% for doc in documents %}
  {{ doc.content }}
{% endfor %}

Answer the given question: {{query}}
Answer:
"""
prompt_builder= PromptBuilder(template=prompt_template)
query_pipeline.add_component("prompt_builder", prompt_builder)
query_pipeline.add_component("generator", generator)
query_pipeline.connect("retriever.documents", "prompt_builder.documents")
query_pipeline.connect("prompt_builder", "generator")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: HuggingFaceAPIGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

In [ ]:
def get_generative_answer(query):

  results =query_pipeline.run({
      "embedder": {"text": query},
      "prompt_builder": {"query": query}
    }
  )
  answer = results["generator"]["replies"][0]
  print(answer)
get_generative_answer("what are TOP 5 CLEAN EATING TIPS")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/inference/_client.py:2245: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


 1. STOCK YOUR KITCHEN. 2. MAKE A MEAL PLAN. 3. CLEAN-EATING COMFORT FOOD. 4. DRINK LOTS OF WATER. 5. CARRY SNACKS EVERYWHERE.


In [ ]:
 question=  "What ingredients would I need to make STRAWBERRY N’ CREAM OVERNIGHT OATS?"
 get_generative_answer(question)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 The ingredients for STRAWBERRY N’ CREAM OVERNIGHT OATS are:

* 1 cup gluten-free rolled oats
* ½ cup coconut milk
* ½ cup almond or cashew milk
* 1 tablespoon chia seeds
* 1 teaspoon vanilla extract
* ½ teaspoon almond extract (optional)
* 1-2 tablespoons maple syrup
* Pinch of salt
* ¾ cup chopped strawberries
* Toppings: sliced strawberries, chia seeds, sliced almonds, shredded coconut

To make the STRAWBERRY N’ CREAM OVERNIGHT OATS, combine oats, milk, chia seeds, vanilla and almond extract, salt and maple syrup in a large bowl. Stir together until well combined. Fold in chopped strawberries. Cover and place in the refrigerator at least 4 hours or overnight. When ready, divide into two cups and top with desired toppings.
